### Imports and functions

In [12]:
import pandas as pd
import numpy as np
from math import sqrt
from datetime import timedelta
import pickle

#data viz imports
import plotly.graph_objs as go
#use this format for working locally 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot, plot_mpl
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#set numbers formatting 
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [13]:
def is_null(df, column):
    '''
    Checks if column has null values, prints the number of nulls in column,
    and returns nulls as a dataframes.
    
    If no nulls prints that there is no null values.
    
    :df: The dateframe that the user wants to check
    :column: The column on the dataframe the user wants to check
    
    Returns number of nulls in column and a new dataframe with nulls.
    '''
    if df[df[column].isnull()].empty:
        print('%s: No Nulls'%(column))
    else:
        print('%s: There are Nulls'%(column))
        return df[df[column].isnull()]

### data loading and cleaning

In [5]:
#historical is past 5 years of data
historical = pd.read_pickle('/Users/GabeKlick/Desktop/Metis/Projects/Project5/trafficdatetime.pkl')

#current data is last 6 months of data
current = pd.read_pickle('/Users/GabeKlick/Desktop/Metis/Projects/Project5/currenttrafficdatetime.pkl')

In [6]:
#Data is store every 10 minutes at each tracker, which is a lot of data!
#lets limit to just the last year, since with time series more recent observations are better predicters
historical = historical[historical.TIME.dt.year >= 2017]


#rename column
historical['NUMBER OF READS'] = historical['NUMBER OF READS                      ']
historical.drop('NUMBER OF READS                      ', axis = 1, inplace = True)

historical.head()

,TIME,REGION_ID,BUS COUNT,SPEED,NUMBER OF READS
3096225,2018-05-03 12:01:31,24,47,22.500,845
3096266,2018-05-03 12:01:31,25,18,29.320,277
3096389,2018-05-03 12:01:31,26,49,27.270,896
3096471,2018-05-03 12:01:31,27,22,27.340,270
3100392,2018-05-03 12:01:31,28,7,39.550,101


In [7]:
#Current and historical need to have the same column names in the same order for concation
current['BUS COUNT'] = current.BUS_COUNT
current['NUMBER OF READS'] = current['NUM_READS']
current = current[['TIME', 'REGION_ID','BUS COUNT', 'NUMBER OF READS','SPEED']]

current.head()

,TIME,REGION_ID,BUS COUNT,NUMBER OF READS,SPEED
0,2018-11-27 15:11:13,27,34,595,30.000
1,2018-11-27 15:11:13,28,12,255,32.730
2,2018-11-27 15:11:13,29,49,491,20.450
3,2018-11-27 15:11:12,1,32,706,21.820
4,2018-11-27 15:11:12,2,56,1220,23.860


In [8]:
#Combining for a little more than a years worth of data
traffic_df = pd.concat([historical, current], axis = 0, ignore_index = True)
print("The sum is equal to the whole: ",len(traffic_df) == len(historical) + len(current))

#check to make sure dups are dropped
print('\nDF with duplicates: ',traffic_df.shape[0])
traffic_df.drop_duplicates(inplace=True)
print('DF without duplicates: ',traffic_df.shape[0])

#sort values and reset index
traffic_df.sort_values('TIME', inplace = True)
traffic_df.reset_index(drop = True, inplace = True)

traffic_df.head()

The sum is equal to the whole:  True

DF with duplicates:  3029568


/Users/GabeKlick/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False




DF without duplicates:  2766598


,BUS COUNT,NUMBER OF READS,REGION_ID,SPEED,TIME
0,12,215,5,25.230,2017-01-01 00:10:26
1,12,230,6,24.550,2017-01-01 00:10:26
2,10,150,1,28.640,2017-01-01 00:10:26
3,6,85,2,29.180,2017-01-01 00:10:26
4,20,288,3,27.950,2017-01-01 00:10:26


In [9]:
#check for nulls
for column in traffic_df:
    is_null(traffic_df, column)

BUS COUNT: No Nulls
NUMBER OF READS: No Nulls
REGION_ID: No Nulls
SPEED: No Nulls
TIME: No Nulls


In [10]:
traffic_df.to_pickle("traffic_df.pkl")

### checking actual vs. predicted

In [14]:
for i in range(1,30):
    model_name = 'region%s_model'%(i)
    metric_name = 'region%s_stats'%(i)
    forecast_name = 'region%s_forecast'%(i)

    with open(model_name, "rb") as f:
        globals()['model%s'%(i)] = pickle.load(f)

    with open(metric_name, "rb") as g:
        print('region ', i)
        metrics = pickle.load(g)
        print('RMSE: ',metrics[-1])
        print('MAPE: ',metrics[1])
    
    with open(forecast_name, "rb") as h:
        globals()['forecast%s'%(i)] = pickle.load(h)

KeyboardInterrupt: 